In [0]:
%run ./init

In [0]:
# import os
# import re
# import cv2
# import pytesseract
# import numpy as np
# import pandas as pd
# import logging
# import json
# import matplotlib.pyplot as plt
# from datetime import datetime

# # ---------------------------------------------------------------------
# # Logger Setup
# # ---------------------------------------------------------------------
# logger = logging.getLogger("timeBreakdownExtractor")
# logger.setLevel(logging.DEBUG)  # Set debug level for detailed logs
# if not logger.handlers:
#     handler = logging.StreamHandler()
#     handler.setFormatter(logging.Formatter("%(levelname)s: %(message)s"))
#     logger.addHandler(handler)

# # ---------------------------------------------------------------------
# # Utility: show_image
# # ---------------------------------------------------------------------
# def show_image(title, img, cmap=None, size=(10,10)):
#     plt.figure(figsize=size)
#     if cmap:
#         plt.imshow(img, cmap=cmap)
#     else:
#         if len(img.shape) == 3:
#             plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#         else:
#             plt.imshow(img, cmap="gray")
#     plt.title(title)
#     plt.axis("off")
#     plt.show()

# # ---------------------------------------------------------------------
# # safe_read_image
# # ---------------------------------------------------------------------
# def safe_read_image(img_path):
#     """Reads an image from a DBFS or local path."""
#     if img_path.startswith("dbfs:"):
#         local_path = img_path.replace("dbfs:", "/dbfs")
#     else:
#         local_path = img_path
#     logger.info(f"Trying to read image from: {local_path}")
#     if not os.path.exists(local_path):
#         logger.error(f"File not found: {local_path}")
#         raise FileNotFoundError(f"File not found: {local_path}")
#     img = cv2.imread(local_path)
#     if img is None:
#         logger.error(f"OpenCV failed to read the image: {local_path}")
#         raise FileNotFoundError(f"OpenCV failed to read the image: {local_path}")
#     logger.info("Image loaded successfully.")
#     return img

# # ---------------------------------------------------------------------
# # preprocess_image
# # ---------------------------------------------------------------------
# def preprocess_image(img, debug=False):
#     """Converts image to grayscale and applies adaptive thresholding."""
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     if debug:
#         show_image("1) Grayscale", gray, cmap="gray")
#     thresh = cv2.adaptiveThreshold(
#         gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#         cv2.THRESH_BINARY, 15, 9
#     )
#     if debug:
#         show_image("2) Thresholded", thresh, cmap="gray")
#     logger.debug("Preprocessing complete.")
#     return thresh

# # ---------------------------------------------------------------------
# # single_box_ocr
# # ---------------------------------------------------------------------
# def single_box_ocr(img, debug=False):
#     """Performs OCR on the entire image region."""
#     h, w = img.shape[:2]
#     roi = img[0:h, 0:w]
#     text = pytesseract.image_to_string(roi, config="--psm 6")
#     logger.debug(f"OCR produced {len(text)} characters.")
#     if debug:
#         preview = "\n".join(text.splitlines()[:5])
#         logger.debug("OCR text preview:\n" + preview)
#     return text

# # ---------------------------------------------------------------------
# # compute_hours
# # ---------------------------------------------------------------------
# def compute_hours(from_time, to_time):
#     """Computes the difference in hours between two times formatted as HH:MM."""
#     fmt = "%H:%M"
#     try:
#         t1 = datetime.strptime(from_time, fmt)
#         t2 = datetime.strptime(to_time, fmt)
#         # Handle midnight crossing: if to_time is less than from_time, add one day.
#         if t2 < t1:
#             t2 = t2.replace(day=t2.day+1)
#         diff = t2 - t1
#         hours = diff.seconds / 3600.0
#         return f"{hours:.2f}"
#     except Exception as e:
#         logger.error(f"Error computing hours: {e}")
#         return "[UNKNOWN]"

# # ---------------------------------------------------------------------
# # parse_operations_description
# # ---------------------------------------------------------------------
# def parse_operations_description(ops_text):
#     """
#     Parses the operations description text into a structured dictionary.
#     This template uses regex to extract key metrics; adjust as needed.
#     """
#     ops_data = {
#         "Depth": {"From": "", "To": ""},
#         "Performance": {"Feet": "", "FPH": ""},
#         "Rotation_Slide": {"Rotate": "", "Slide": ""},
#         "Rotation_Time": {"Rotate Time": "", "Slide Time": ""},
#         "GPM": "",
#         "MTR RPM": "",
#         "SPP": "",
#         "DIFF": "",
#         "WOB": "",
#         "ROT RPM": "",
#         "ON BTM TRQ": "",
#         "OFF BTM TRQ": "",
#         "GAS": {"Units": "", "Flare": ""},
#         "MW": {"In": "", "Out": ""},
#         "Targets": [],
#         "Observations": []
#     }
#     lines = ops_text.splitlines()
#     logger.debug(f"Parsing operations description with {len(lines)} lines.")
#     for line in lines:
#         l = line.strip()
#         if not l:
#             continue

#         depth_match = re.search(r"F/\s*([\d,']+)\s*T/\s*([\d,']+)", l, re.IGNORECASE)
#         if depth_match:
#             ops_data["Depth"]["From"] = depth_match.group(1)
#             ops_data["Depth"]["To"] = depth_match.group(2)
#         perf_match = re.search(r"\(([\d,']+)\s*@\s*(\d+)\s*FPH\)", l, re.IGNORECASE)
#         if perf_match:
#             ops_data["Performance"]["Feet"] = perf_match.group(1)
#             ops_data["Performance"]["FPH"] = perf_match.group(2)
#         rs_match = re.search(r"ROTATE\s*([\d.]+%)\s*/\s*SLIDE\s*([\d.]+%)", l, re.IGNORECASE)
#         if rs_match:
#             ops_data["Rotation_Slide"]["Rotate"] = rs_match.group(1)
#             ops_data["Rotation_Slide"]["Slide"] = rs_match.group(2)
#         rt_match = re.search(r"ROTATE\s*TIME\s*([\d.]+%)\s*/\s*SLIDE\s*TIME\s*([\d.]+%)", l, re.IGNORECASE)
#         if rt_match:
#             ops_data["Rotation_Time"]["Rotate Time"] = rt_match.group(1)
#             ops_data["Rotation_Time"]["Slide Time"] = rt_match.group(2)
#         gpm = re.search(r"GPM:\s*(\d+)", l, re.IGNORECASE)
#         if gpm:
#             ops_data["GPM"] = gpm.group(1)
#         mtr = re.search(r"MTR\s*RPM:\s*(\d+)", l, re.IGNORECASE)
#         if mtr:
#             ops_data["MTR RPM"] = mtr.group(1)
#         spp = re.search(r"SPP:\s*(\d+)", l, re.IGNORECASE)
#         if spp:
#             ops_data["SPP"] = spp.group(1)
#         diff = re.search(r"DIFF:\s*([\d\-]+)", l, re.IGNORECASE)
#         if diff:
#             ops_data["DIFF"] = diff.group(1)
#         wob = re.search(r"WOB:\s*(\d+)", l, re.IGNORECASE)
#         if wob:
#             ops_data["WOB"] = wob.group(1)
#         rot_rpm = re.search(r"ROT\s*RPM:\s*(\d+)", l, re.IGNORECASE)
#         if rot_rpm:
#             ops_data["ROT RPM"] = rot_rpm.group(1)
#         on_btm = re.search(r"ON\s*BTM\s*TRQ[:;]?\s*([\d\-K]+)", l, re.IGNORECASE)
#         if on_btm:
#             ops_data["ON BTM TRQ"] = on_btm.group(1)
#         off_btm = re.search(r"OFF\s*BTM\s*TRQ[:;]?\s*([\d\-K]+)", l, re.IGNORECASE)
#         if off_btm:
#             ops_data["OFF BTM TRQ"] = off_btm.group(1)
#         gas_units = re.search(r"GAS:\s*([\d,]+)\s*UNITS", l, re.IGNORECASE)
#         if gas_units:
#             ops_data["GAS"]["Units"] = gas_units.group(1)
#         flare = re.search(r"(NO\s*FLARE|FLARE\s*ON|FLARE\s*\S+)", l, re.IGNORECASE)
#         if flare:
#             ops_data["GAS"]["Flare"] = flare.group(1)
#         mw_in = re.search(r"MW\s*IN\s*([\d.]+)", l, re.IGNORECASE)
#         if mw_in:
#             ops_data["MW"]["In"] = mw_in.group(1)
#         mw_out = re.search(r"OUT\s*([\d.]+)", l, re.IGNORECASE)
#         if mw_out:
#             ops_data["MW"]["Out"] = mw_out.group(1)
#         if re.search(r"TARGET\s*#\d+", l, re.IGNORECASE):
#             ops_data["Targets"].append(l)
#         if not re.search(r"(F/|ROTATE|GPM:|MTR RPM:|SPP:)", l, re.IGNORECASE):
#             ops_data["Observations"].append(l)
#     ops_data["Observations"] = list(dict.fromkeys(ops_data["Observations"]))
#     return ops_data

# # ---------------------------------------------------------------------
# # parse_time_breakdown_by_lines
# # ---------------------------------------------------------------------
# def parse_time_breakdown_by_lines(ocr_text):
#     """
#     Splits the OCR text into lines, groups them into row groups when a line starts with a time stamp,
#     and then parses each group into a row dictionary.
#     The header line is expected to contain tokens in the order:
#       From, To, Depth Start, Depth End, then Phase and Activity.
#     Hours is computed from the time difference.
#     """
#     lines = ocr_text.splitlines()
#     logger.debug(f"Total OCR lines: {len(lines)}")
    
#     # Remove header lines containing fixed keywords.
#     header_keywords = ["Time Period", "From To Hours", "Depth Phase", "Operations Description"]
#     filtered_lines = [line.strip() for line in lines if line.strip() and not any(kw in line for kw in header_keywords)]
#     logger.debug(f"Filtered lines count: {len(filtered_lines)}")
    
#     # Group lines: a new group starts when a line starts with a time stamp (HH:MM).
#     groups = []
#     current_group = []
#     time_pattern = re.compile(r"^\d{2}:\d{2}")
#     for line in filtered_lines:
#         if time_pattern.match(line):
#             if current_group:
#                 groups.append(current_group)
#                 logger.debug(f"Created group with {len(current_group)} lines.")
#             current_group = [line]
#         else:
#             if current_group:
#                 current_group.append(line)
#     if current_group:
#         groups.append(current_group)
#         logger.debug(f"Final group added with {len(current_group)} lines.")
#     logger.info(f"Total row groups detected: {len(groups)}")
    
#     parsed_rows = []
#     for idx, group in enumerate(groups):
#         header_line = group[0]
#         tokens = re.split(r"\s+", header_line)
#         logger.debug(f"Row {idx} header tokens: {tokens}")
#         if len(tokens) < 5:
#             logger.warning(f"Row {idx} skipped due to insufficient tokens: {tokens}")
#             continue
        
#         from_time = tokens[0]
#         to_time = tokens[1]
#         # Compute Hours from time difference.
#         hours = compute_hours(from_time, to_time)
#         depth_start = tokens[2]
#         depth_end = tokens[3]
        
#         # Search tokens[4:] for the activity token (e.g. starting with "DR-")
#         activity = None
#         activity_idx = None
#         for i, token in enumerate(tokens[4:], start=4):
#             if re.match(r"DR-\S+", token, re.IGNORECASE):
#                 activity = token
#                 activity_idx = i
#                 break
#         if activity is None:
#             logger.warning(f"Row {idx} could not find an activity token in {tokens[4:]}")
#             activity = "[UNKNOWN]"
#             phase = " ".join(tokens[4:])
#             ops_tokens = group[1:] if len(group) > 1 else []
#         else:
#             phase = " ".join(tokens[4:activity_idx])
#             ops_tokens = tokens[activity_idx+1:]
#             # Also include any additional lines from the group as part of operations.
#             if len(group) > 1:
#                 ops_tokens.extend(" ".join(group[1:]).split())
#         ops_text = " ".join(ops_tokens)
#         logger.debug(f"Row {idx} parsed header: From={from_time}, To={to_time}, Hours={hours}, Depth Start={depth_start}, Depth End={depth_end}, Phase='{phase}', Activity='{activity}'")
#         logger.debug(f"Row {idx} operations text: {ops_text[:100]}...")
#         ops_description = parse_operations_description(ops_text)
        
#         row_dict = {
#             "From": from_time,
#             "To": to_time,
#             "Hours": hours,
#             "Depth Start": depth_start,
#             "Depth End": depth_end,
#             "Phase": phase.strip(),
#             "Activity": activity.strip(),
#             "Operations Description": ops_description
#         }
#         parsed_rows.append(row_dict)
    
#     logger.info(f"Total parsed rows: {len(parsed_rows)}")
#     return parsed_rows

# # ---------------------------------------------------------------------
# # main_time_breakdown_pipeline
# # ---------------------------------------------------------------------
# def main_time_breakdown_pipeline():
#     # Set your image path (adjust as needed)
#     img_path = "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_14.png"
#     try:
#         img = safe_read_image(img_path)
#     except Exception as e:
#         logger.error(e)
#         return

#     # Preprocess the image.
#     thresh_img = preprocess_image(img, debug=True)

#     # Perform OCR on the entire image.
#     full_text = single_box_ocr(thresh_img, debug=True)

#     # Parse the OCR text into rows.
#     time_breakdown_list = parse_time_breakdown_by_lines(full_text)
#     if not time_breakdown_list:
#         logger.error("No rows were detected. Please check the OCR text and header format.")
#         return

#     # Build final JSON output.
#     final_output = {"TIME BREAKDOWN": time_breakdown_list}
#     logger.info("===== FINAL TIME BREAKDOWN DATA =====")
#     logger.info(json.dumps(final_output, indent=4))

#     # Create a DataFrame (nested fields flattened).
#     df = pd.json_normalize(final_output["TIME BREAKDOWN"])
#     print("----- Extracted Time Breakdown DataFrame -----")
#     print(df)

#     # Save results as JSON and CSV.
#     output_folder = "dbfs:/mnt/mini-proj-dd/final_time_breakdown_results"
#     local_folder = output_folder.replace("dbfs:", "/dbfs")
#     os.makedirs(local_folder, exist_ok=True)
#     out_json = os.path.join(local_folder, "time_breakdown_data.json")
#     with open(out_json, "w") as f:
#         json.dump(final_output, f, indent=4)
#     logger.info(f"JSON saved to {out_json}")
#     out_csv = os.path.join(local_folder, "time_breakdown_data.csv")
#     df.to_csv(out_csv, index=False)
#     logger.info(f"CSV saved to {out_csv}")

# if __name__ == "__main__":
#     main_time_breakdown_pipeline()


In [0]:
import os
import re
import cv2
import pytesseract
import numpy as np
import pandas as pd
import logging
import json
import matplotlib.pyplot as plt

# ---------------------------------------------------------------------
# Logger Setup
# ---------------------------------------------------------------------
logger = logging.getLogger("timeBreakdownExtractor")
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("%(levelname)s: %(message)s"))
    logger.addHandler(handler)

# ---------------------------------------------------------------------
# Utility: show_image
# ---------------------------------------------------------------------
def show_image(title, img, cmap=None, size=(10,10)):
    plt.figure(figsize=size)
    if cmap:
        plt.imshow(img, cmap=cmap)
    else:
        if len(img.shape) == 3:
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            plt.imshow(img, cmap="gray")
    plt.title(title)
    plt.axis("off")
    plt.show()

# ---------------------------------------------------------------------
# safe_read_image
# ---------------------------------------------------------------------
def safe_read_image(img_path):
    """Reads an image from a DBFS or local path."""
    if img_path.startswith("dbfs:"):
        local_path = img_path.replace("dbfs:", "/dbfs")
    else:
        local_path = img_path
    logger.info(f"Trying to read image from: {local_path}")
    if not os.path.exists(local_path):
        raise FileNotFoundError(f"File not found: {local_path}")
    img = cv2.imread(local_path)
    if img is None:
        raise FileNotFoundError(f"OpenCV failed to read the image: {local_path}")
    return img

# ---------------------------------------------------------------------
# preprocess_image
# ---------------------------------------------------------------------
def preprocess_image(img, debug=False):
    """Converts image to grayscale and applies adaptive thresholding."""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if debug:
        show_image("1) Grayscale", gray, cmap="gray")
    thresh = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 15, 9
    )
    if debug:
        show_image("2) Thresholded", thresh, cmap="gray")
    return thresh

# ---------------------------------------------------------------------
# single_box_ocr
# ---------------------------------------------------------------------
def single_box_ocr(img, debug=False):
    """Performs OCR on the entire image region."""
    h, w = img.shape[:2]
    roi = img[0:h, 0:w]
    text = pytesseract.image_to_string(roi, config="--psm 6")
    if debug:
        logger.info("OCR text:\n" + text)
    return text

# ---------------------------------------------------------------------
# parse_time_breakdown_by_lines
# ---------------------------------------------------------------------
def parse_time_breakdown_by_lines(ocr_text):
    """
    Splits the OCR text line-by-line and groups lines into rows.
    A new row is assumed to begin when a line starts with a time stamp (e.g. "06:00").
    Lines that are clearly headers (like "From To Hours") are skipped.
    Returns a list of dictionaries, one per row.
    """
    lines = ocr_text.splitlines()

    # Filter out obvious header lines.
    header_keywords = ["Time Period", "From To Hours", "Depth Phase", "Operations Description"]
    filtered_lines = []
    for line in lines:
        ls = line.strip()
        if ls and not any(kw in ls for kw in header_keywords):
            filtered_lines.append(ls)
    
    rows = []
    current_group = []
    time_pattern = re.compile(r"^\d{2}:\d{2}")  # line starting with HH:MM

    for line in filtered_lines:
        if time_pattern.match(line):
            # If we already have a group, save it.
            if current_group:
                rows.append("\n".join(current_group))
            current_group = [line]
        else:
            # Append to current group.
            if current_group:
                current_group.append(line)
    if current_group:
        rows.append("\n".join(current_group))
    
    parsed_rows = []
    # Now process each group (row).
    for group in rows:
        # Split group into lines and tokens.
        group_lines = group.splitlines()
        if not group_lines:
            continue
        header_line = group_lines[0]
        # Split header_line into tokens (whitespace split)
        tokens = re.split(r"\s+", header_line.strip())
        # We assume header tokens are in order: From, To, Hours, Depth Start, Depth End, then remainder for Phase and Activity.
        # Adjust expected count if needed.
        if len(tokens) < 6:
            continue  # Not enough data, skip.
        from_time = tokens[0]
        to_time = tokens[1]
        hours = tokens[2]
        depth_start = tokens[3]
        depth_end = tokens[4]
        # The rest of tokens in header: try to separate Phase and Activity.
        remainder = tokens[5:]
        phase = " ".join(remainder[:-1]) if len(remainder) > 1 else remainder[0]
        activity = remainder[-1] if len(remainder) > 1 else "[UNKNOWN]"
        # The rest of the group (if any) becomes the raw operations text.
        ops_text = "\n".join(group_lines[1:]) if len(group_lines) > 1 else ""
        ops_description = parse_operations_description(ops_text)
        parsed_row = {
            "From": from_time,
            "To": to_time,
            "Hours": hours,
            "Depth Start": depth_start,
            "Depth End": depth_end,
            "Phase": phase,
            "Activity": activity,
            "Operations Description": ops_description
        }
        parsed_rows.append(parsed_row)
    
    return parsed_rows

# ---------------------------------------------------------------------
# parse_operations_description
# ---------------------------------------------------------------------
def parse_operations_description(ops_text):
    """
    Parses the operations description text into a structured dictionary.
    This template uses regex to extract key metrics. Adjust as needed.
    """
    ops_data = {
        "Depth": {"From": "", "To": ""},
        "Performance": {"Feet": "", "FPH": ""},
        "Rotation_Slide": {"Rotate": "", "Slide": ""},
        "Rotation_Time": {"Rotate Time": "", "Slide Time": ""},
        "GPM": "",
        "MTR RPM": "",
        "SPP": "",
        "DIFF": "",
        "WOB": "",
        "ROT RPM": "",
        "ON BTM TRQ": "",
        "OFF BTM TRQ": "",
        "GAS": {"Units": "", "Flare": ""},
        "MW": {"In": "", "Out": ""},
        "Targets": [],
        "Observations": []
    }

    lines = ops_text.splitlines()
    for line in lines:
        l = line.strip()
        if not l:
            continue

        # Depth extraction: e.g. "F/ 17,469' T/ 18,163'"
        depth_match = re.search(r"F/\s*([\d,']+)\s*T/\s*([\d,']+)", l, re.IGNORECASE)
        if depth_match:
            ops_data["Depth"]["From"] = depth_match.group(1)
            ops_data["Depth"]["To"] = depth_match.group(2)

        # Performance: e.g. "(694' @ 115 FPH)"
        perf_match = re.search(r"\(([\d,']+)\s*@\s*(\d+)\s*FPH\)", l, re.IGNORECASE)
        if perf_match:
            ops_data["Performance"]["Feet"] = perf_match.group(1)
            ops_data["Performance"]["FPH"] = perf_match.group(2)

        # Rotation/Slide percentages.
        rs_match = re.search(r"ROTATE\s*([\d.]+%)\s*/\s*SLIDE\s*([\d.]+%)", l, re.IGNORECASE)
        if rs_match:
            ops_data["Rotation_Slide"]["Rotate"] = rs_match.group(1)
            ops_data["Rotation_Slide"]["Slide"] = rs_match.group(2)

        # Rotation Time percentages.
        rt_match = re.search(r"ROTATE\s*TIME\s*([\d.]+%)\s*/\s*SLIDE\s*TIME\s*([\d.]+%)", l, re.IGNORECASE)
        if rt_match:
            ops_data["Rotation_Time"]["Rotate Time"] = rt_match.group(1)
            ops_data["Rotation_Time"]["Slide Time"] = rt_match.group(2)

        # Numeric parameters.
        gpm = re.search(r"GPM:\s*(\d+)", l, re.IGNORECASE)
        if gpm:
            ops_data["GPM"] = gpm.group(1)
        mtr = re.search(r"MTR\s*RPM:\s*(\d+)", l, re.IGNORECASE)
        if mtr:
            ops_data["MTR RPM"] = mtr.group(1)
        spp = re.search(r"SPP:\s*(\d+)", l, re.IGNORECASE)
        if spp:
            ops_data["SPP"] = spp.group(1)
        diff = re.search(r"DIFF:\s*([\d\-]+)", l, re.IGNORECASE)
        if diff:
            ops_data["DIFF"] = diff.group(1)
        wob = re.search(r"WOB:\s*(\d+)", l, re.IGNORECASE)
        if wob:
            ops_data["WOB"] = wob.group(1)
        rot_rpm = re.search(r"ROT\s*RPM:\s*(\d+)", l, re.IGNORECASE)
        if rot_rpm:
            ops_data["ROT RPM"] = rot_rpm.group(1)
        on_btm = re.search(r"ON\s*BTM\s*TRQ[:;]?\s*([\d\-K]+)", l, re.IGNORECASE)
        if on_btm:
            ops_data["ON BTM TRQ"] = on_btm.group(1)
        off_btm = re.search(r"OFF\s*BTM\s*TRQ[:;]?\s*([\d\-K]+)", l, re.IGNORECASE)
        if off_btm:
            ops_data["OFF BTM TRQ"] = off_btm.group(1)
        gas_units = re.search(r"GAS:\s*([\d,]+)\s*UNITS", l, re.IGNORECASE)
        if gas_units:
            ops_data["GAS"]["Units"] = gas_units.group(1)
        flare = re.search(r"(NO\s*FLARE|FLARE\s*ON|FLARE\s*\S+)", l, re.IGNORECASE)
        if flare:
            ops_data["GAS"]["Flare"] = flare.group(1)
        mw_in = re.search(r"MW\s*IN\s*([\d.]+)", l, re.IGNORECASE)
        if mw_in:
            ops_data["MW"]["In"] = mw_in.group(1)
        mw_out = re.search(r"OUT\s*([\d.]+)", l, re.IGNORECASE)
        if mw_out:
            ops_data["MW"]["Out"] = mw_out.group(1)

        # Capture Targets.
        if re.search(r"TARGET\s*#\d+", l, re.IGNORECASE):
            ops_data["Targets"].append(l)
        # Other lines become Observations if they don't match key patterns.
        if not re.search(r"(F/|ROTATE|GPM:|MTR RPM:|SPP:)", l, re.IGNORECASE):
            ops_data["Observations"].append(l)

    # Remove duplicate observations.
    ops_data["Observations"] = list(dict.fromkeys(ops_data["Observations"]))
    return ops_data

# ---------------------------------------------------------------------
# main_time_breakdown_pipeline
# ---------------------------------------------------------------------
def main_time_breakdown_pipeline():
    # Set your image path (adjust as needed)
    img_path = "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_14.png"
    try:
        img = safe_read_image(img_path)
        logger.info("Image loaded successfully.")
    except Exception as e:
        logger.error(e)
        return

    # Preprocess the image.
    thresh_img = preprocess_image(img, debug=True)

    # Perform OCR on the entire region.
    full_text = single_box_ocr(thresh_img, debug=True)

    # Dynamically split the OCR text into multiple rows using line-by-line grouping.
    time_breakdown_list = parse_time_breakdown_by_lines(full_text)
    if not time_breakdown_list:
        logger.error("No rows were detected. Please check the OCR output and header format.")
        return

    # Build final JSON output.
    final_output = {"TIME BREAKDOWN": time_breakdown_list}
    logger.info("===== FINAL TIME BREAKDOWN DATA =====")
    logger.info(json.dumps(final_output, indent=4))

    # Create a DataFrame (nested fields flattened).
    df = pd.json_normalize(final_output["TIME BREAKDOWN"])
    print("----- Extracted Time Breakdown DataFrame -----")
    print(df)

    # Save results as JSON and CSV.
    output_folder = "dbfs:/mnt/mini-proj-dd/final_time_breakdown_results"
    local_folder = output_folder.replace("dbfs:", "/dbfs")
    os.makedirs(local_folder, exist_ok=True)
    out_json = os.path.join(local_folder, "time_breakdown_data.json")
    with open(out_json, "w") as f:
        json.dump(final_output, f, indent=4)
    logger.info(f"JSON saved to {out_json}")
    out_csv = os.path.join(local_folder, "time_breakdown_data.csv")
    df.to_csv(out_csv, index=False)
    logger.info(f"CSV saved to {out_csv}")

if __name__ == "__main__":
    main_time_breakdown_pipeline()


In [0]:
# import os
# import re
# import cv2
# import pytesseract
# import numpy as np
# import pandas as pd
# import logging
# import json
# import matplotlib.pyplot as plt

# # ---------------------------------------------------------------------
# # Logger Setup
# # ---------------------------------------------------------------------
# logger = logging.getLogger("timeBreakdownExtractor")
# logger.setLevel(logging.DEBUG)  # Set to DEBUG to see detailed logs.
# if not logger.handlers:
#     handler = logging.StreamHandler()
#     handler.setFormatter(logging.Formatter("%(levelname)s: %(message)s"))
#     logger.addHandler(handler)

# # ---------------------------------------------------------------------
# # Utility: show_image
# # ---------------------------------------------------------------------
# def show_image(title, img, cmap=None, size=(10,10)):
#     plt.figure(figsize=size)
#     if cmap:
#         plt.imshow(img, cmap=cmap)
#     else:
#         if len(img.shape) == 3:
#             plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#         else:
#             plt.imshow(img, cmap="gray")
#     plt.title(title)
#     plt.axis("off")
#     plt.show()

# # ---------------------------------------------------------------------
# # safe_read_image
# # ---------------------------------------------------------------------
# def safe_read_image(img_path):
#     """Reads an image from a DBFS or local path."""
#     if img_path.startswith("dbfs:"):
#         local_path = img_path.replace("dbfs:", "/dbfs")
#     else:
#         local_path = img_path
#     logger.info(f"Trying to read image from: {local_path}")
#     if not os.path.exists(local_path):
#         logger.error(f"File not found: {local_path}")
#         raise FileNotFoundError(f"File not found: {local_path}")
#     img = cv2.imread(local_path)
#     if img is None:
#         logger.error(f"OpenCV failed to read the image: {local_path}")
#         raise FileNotFoundError(f"OpenCV failed to read the image: {local_path}")
#     logger.info("Image loaded successfully.")
#     return img

# # ---------------------------------------------------------------------
# # preprocess_image
# # ---------------------------------------------------------------------
# def preprocess_image(img, debug=False):
#     """Converts image to grayscale and applies adaptive thresholding."""
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     if debug:
#         show_image("1) Grayscale", gray, cmap="gray")
#     thresh = cv2.adaptiveThreshold(
#         gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#         cv2.THRESH_BINARY, 15, 9
#     )
#     if debug:
#         show_image("2) Thresholded", thresh, cmap="gray")
#     logger.debug("Preprocessing complete.")
#     return thresh

# # ---------------------------------------------------------------------
# # single_box_ocr
# # ---------------------------------------------------------------------
# def single_box_ocr(img, debug=False):
#     """Performs OCR on the entire image region."""
#     h, w = img.shape[:2]
#     roi = img[0:h, 0:w]
#     text = pytesseract.image_to_string(roi, config="--psm 6")
#     logger.debug(f"OCR produced {len(text)} characters.")
#     if debug:
#         # Log the first few lines of OCR output
#         logger.debug("OCR text preview:\n" + "\n".join(text.splitlines()[:5]))
#     return text

# # ---------------------------------------------------------------------
# # parse_operations_description
# # ---------------------------------------------------------------------
# def parse_operations_description(ops_text):
#     """
#     Parses the operations description text into a structured dictionary.
#     Adjust regexes as needed.
#     """
#     ops_data = {
#         "Depth": {"From": "", "To": ""},
#         "Performance": {"Feet": "", "FPH": ""},
#         "Rotation_Slide": {"Rotate": "", "Slide": ""},
#         "Rotation_Time": {"Rotate Time": "", "Slide Time": ""},
#         "GPM": "",
#         "MTR RPM": "",
#         "SPP": "",
#         "DIFF": "",
#         "WOB": "",
#         "ROT RPM": "",
#         "ON BTM TRQ": "",
#         "OFF BTM TRQ": "",
#         "GAS": {"Units": "", "Flare": ""},
#         "MW": {"In": "", "Out": ""},
#         "Targets": [],
#         "Observations": []
#     }
    
#     lines = ops_text.splitlines()
#     logger.debug(f"Parsing operations description with {len(lines)} lines.")
#     for line in lines:
#         l = line.strip()
#         if not l:
#             continue

#         # Depth extraction: e.g. "F/ 17,469' T/ 18,163'"
#         depth_match = re.search(r"F/\s*([\d,']+)\s*T/\s*([\d,']+)", l, re.IGNORECASE)
#         if depth_match:
#             ops_data["Depth"]["From"] = depth_match.group(1)
#             ops_data["Depth"]["To"] = depth_match.group(2)
#         # Performance extraction: e.g. "(694' @ 115 FPH)"
#         perf_match = re.search(r"\(([\d,']+)\s*@\s*(\d+)\s*FPH\)", l, re.IGNORECASE)
#         if perf_match:
#             ops_data["Performance"]["Feet"] = perf_match.group(1)
#             ops_data["Performance"]["FPH"] = perf_match.group(2)
#         # Rotation/Slide percentages.
#         rs_match = re.search(r"ROTATE\s*([\d.]+%)\s*/\s*SLIDE\s*([\d.]+%)", l, re.IGNORECASE)
#         if rs_match:
#             ops_data["Rotation_Slide"]["Rotate"] = rs_match.group(1)
#             ops_data["Rotation_Slide"]["Slide"] = rs_match.group(2)
#         # Rotation Time percentages.
#         rt_match = re.search(r"ROTATE\s*TIME\s*([\d.]+%)\s*/\s*SLIDE\s*TIME\s*([\d.]+%)", l, re.IGNORECASE)
#         if rt_match:
#             ops_data["Rotation_Time"]["Rotate Time"] = rt_match.group(1)
#             ops_data["Rotation_Time"]["Slide Time"] = rt_match.group(2)
#         # Numeric parameters.
#         gpm = re.search(r"GPM:\s*(\d+)", l, re.IGNORECASE)
#         if gpm:
#             ops_data["GPM"] = gpm.group(1)
#         mtr = re.search(r"MTR\s*RPM:\s*(\d+)", l, re.IGNORECASE)
#         if mtr:
#             ops_data["MTR RPM"] = mtr.group(1)
#         spp = re.search(r"SPP:\s*(\d+)", l, re.IGNORECASE)
#         if spp:
#             ops_data["SPP"] = spp.group(1)
#         diff = re.search(r"DIFF:\s*([\d\-]+)", l, re.IGNORECASE)
#         if diff:
#             ops_data["DIFF"] = diff.group(1)
#         wob = re.search(r"WOB:\s*(\d+)", l, re.IGNORECASE)
#         if wob:
#             ops_data["WOB"] = wob.group(1)
#         rot_rpm = re.search(r"ROT\s*RPM:\s*(\d+)", l, re.IGNORECASE)
#         if rot_rpm:
#             ops_data["ROT RPM"] = rot_rpm.group(1)
#         on_btm = re.search(r"ON\s*BTM\s*TRQ[:;]?\s*([\d\-K]+)", l, re.IGNORECASE)
#         if on_btm:
#             ops_data["ON BTM TRQ"] = on_btm.group(1)
#         off_btm = re.search(r"OFF\s*BTM\s*TRQ[:;]?\s*([\d\-K]+)", l, re.IGNORECASE)
#         if off_btm:
#             ops_data["OFF BTM TRQ"] = off_btm.group(1)
#         gas_units = re.search(r"GAS:\s*([\d,]+)\s*UNITS", l, re.IGNORECASE)
#         if gas_units:
#             ops_data["GAS"]["Units"] = gas_units.group(1)
#         flare = re.search(r"(NO\s*FLARE|FLARE\s*ON|FLARE\s*\S+)", l, re.IGNORECASE)
#         if flare:
#             ops_data["GAS"]["Flare"] = flare.group(1)
#         mw_in = re.search(r"MW\s*IN\s*([\d.]+)", l, re.IGNORECASE)
#         if mw_in:
#             ops_data["MW"]["In"] = mw_in.group(1)
#         mw_out = re.search(r"OUT\s*([\d.]+)", l, re.IGNORECASE)
#         if mw_out:
#             ops_data["MW"]["Out"] = mw_out.group(1)
#         # Capture Targets.
#         if re.search(r"TARGET\s*#\d+", l, re.IGNORECASE):
#             ops_data["Targets"].append(l)
#         # Other lines become Observations if not matching key patterns.
#         if not re.search(r"(F/|ROTATE|GPM:|MTR RPM:|SPP:)", l, re.IGNORECASE):
#             ops_data["Observations"].append(l)

#     # Remove duplicate observations.
#     ops_data["Observations"] = list(dict.fromkeys(ops_data["Observations"]))
#     return ops_data

# # ---------------------------------------------------------------------
# # parse_time_breakdown_by_lines
# # ---------------------------------------------------------------------
# def parse_time_breakdown_by_lines(ocr_text):
#     """
#     Splits the OCR text line-by-line and groups lines into rows.
#     A new row is assumed when a line starts with a time stamp (e.g., "06:00").
#     Logs the number of lines, groups, and details of each group.
#     Returns a list of dictionaries for each row.
#     """
#     lines = ocr_text.splitlines()
#     logger.debug(f"Total OCR lines: {len(lines)}")
    
#     # Filter out header lines.
#     header_keywords = ["Time Period", "From To Hours", "Depth Phase", "Operations Description"]
#     filtered_lines = []
#     for line in lines:
#         ls = line.strip()
#         if ls and not any(kw in ls for kw in header_keywords):
#             filtered_lines.append(ls)
#     logger.debug(f"Filtered OCR lines: {len(filtered_lines)}")
    
#     # Group lines by detecting new row when a line starts with HH:MM.
#     rows_grouped = []
#     current_group = []
#     time_pattern = re.compile(r"^\d{2}:\d{2}")
    
#     for line in filtered_lines:
#         if time_pattern.match(line):
#             if current_group:
#                 rows_grouped.append(current_group)
#                 logger.debug(f"New row group created with {len(current_group)} lines.")
#             current_group = [line]
#         else:
#             if current_group:
#                 current_group.append(line)
#     if current_group:
#         rows_grouped.append(current_group)
#         logger.debug(f"Final row group added with {len(current_group)} lines.")
#     logger.info(f"Total row groups detected: {len(rows_grouped)}")
    
#     # Process each group.
#     parsed_rows = []
#     for idx, group in enumerate(rows_grouped):
#         header_line = group[0]
#         tokens = re.split(r"\s+", header_line.strip())
#         if len(tokens) < 6:
#             logger.warning(f"Skipping row group {idx} due to insufficient tokens: {tokens}")
#             continue
#         from_time = tokens[0]
#         to_time = tokens[1]
#         hours = tokens[2]
#         depth_start = tokens[3]
#         depth_end = tokens[4]
#         remainder = tokens[5:]
#         phase = " ".join(remainder[:-1]) if len(remainder) > 1 else remainder[0]
#         activity = remainder[-1] if len(remainder) > 1 else "[UNKNOWN]"
#         ops_text = "\n".join(group[1:]) if len(group) > 1 else ""
#         ops_description = parse_operations_description(ops_text)
#         row_dict = {
#             "From": from_time,
#             "To": to_time,
#             "Hours": hours,
#             "Depth Start": depth_start,
#             "Depth End": depth_end,
#             "Phase": phase,
#             "Activity": activity,
#             "Operations Description": ops_description
#         }
#         logger.debug(f"Parsed row {idx}: {row_dict}")
#         parsed_rows.append(row_dict)
    
#     logger.info(f"Total parsed rows: {len(parsed_rows)}")
#     return parsed_rows

# # ---------------------------------------------------------------------
# # main_time_breakdown_pipeline
# # ---------------------------------------------------------------------
# def main_time_breakdown_pipeline():
#     # Set your image path.
#     img_path = "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_14.png"
#     try:
#         img = safe_read_image(img_path)
#     except Exception as e:
#         logger.error(e)
#         return

#     # Preprocess the image.
#     thresh_img = preprocess_image(img, debug=True)

#     # Perform OCR on the entire region.
#     full_text = single_box_ocr(thresh_img, debug=True)

#     # Parse the OCR text into rows.
#     time_breakdown_list = parse_time_breakdown_by_lines(full_text)
#     if not time_breakdown_list:
#         logger.error("No rows were detected. Check the OCR text and header format.")
#         return

#     # Build final JSON output.
#     final_output = {"TIME BREAKDOWN": time_breakdown_list}
#     logger.info("===== FINAL TIME BREAKDOWN DATA =====")
#     logger.info(json.dumps(final_output, indent=4))

#     # Create a DataFrame (nested fields will be flattened).
#     df = pd.json_normalize(final_output["TIME BREAKDOWN"])
#     print("----- Extracted Time Breakdown DataFrame -----")
#     print(df)

#     # Save results as JSON and CSV.
#     output_folder = "dbfs:/mnt/mini-proj-dd/final_time_breakdown_results"
#     local_folder = output_folder.replace("dbfs:", "/dbfs")
#     os.makedirs(local_folder, exist_ok=True)
#     out_json = os.path.join(local_folder, "time_breakdown_data.json")
#     with open(out_json, "w") as f:
#         json.dump(final_output, f, indent=4)
#     logger.info(f"JSON saved to {out_json}")
#     out_csv = os.path.join(local_folder, "time_breakdown_data.csv")
#     df.to_csv(out_csv, index=False)
#     logger.info(f"CSV saved to {out_csv}")

# if __name__ == "__main__":
#     main_time_breakdown_pipeline()


In [0]:
import os
import re
import cv2
import pytesseract
import numpy as np
import pandas as pd
import logging
import json
import matplotlib.pyplot as plt

# ---------------------------------------------------------------------
# Logger Setup
# ---------------------------------------------------------------------
logger = logging.getLogger("timeBreakdownExtractor")
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("%(levelname)s: %(message)s"))
    logger.addHandler(handler)

# ---------------------------------------------------------------------
# Utility: show_image
# ---------------------------------------------------------------------
def show_image(title, img, cmap=None, size=(10,10)):
    plt.figure(figsize=size)
    if cmap:
        plt.imshow(img, cmap=cmap)
    else:
        if len(img.shape) == 3:
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            plt.imshow(img, cmap="gray")
    plt.title(title)
    plt.axis("off")
    plt.show()

# ---------------------------------------------------------------------
# safe_read_image
# ---------------------------------------------------------------------
def safe_read_image(img_path):
    """
    Reads an image from a DBFS or local path.
    """
    if img_path.startswith("dbfs:"):
        local_path = img_path.replace("dbfs:", "/dbfs")
    else:
        local_path = img_path
    logger.info(f"Trying to read image from: {local_path}")
    if not os.path.exists(local_path):
        raise FileNotFoundError(f"File not found: {local_path}")
    img = cv2.imread(local_path)
    if img is None:
        raise FileNotFoundError(f"OpenCV failed to read the image: {local_path}")
    return img

# ---------------------------------------------------------------------
# preprocess_image
# ---------------------------------------------------------------------
def preprocess_image(img, debug=False):
    """
    Converts image to grayscale and applies adaptive thresholding.
    """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if debug:
        show_image("1) Grayscale", gray, cmap="gray")
    thresh = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 15, 9
    )
    if debug:
        show_image("2) Thresholded", thresh, cmap="gray")
    return thresh

# ---------------------------------------------------------------------
# single_box_ocr
# ---------------------------------------------------------------------
def single_box_ocr(img, debug=False):
    """
    Performs OCR on the entire image (or table region) as one chunk.
    """
    h, w = img.shape[:2]
    roi = img[0:h, 0:w]
    text = pytesseract.image_to_string(roi, config="--psm 6")
    if debug:
        logger.info("OCR text:\n" + text)
    return text

# ---------------------------------------------------------------------
# parse_operations_description
# ---------------------------------------------------------------------
def parse_operations_description(ops_text):
    """
    Parses the operations description text into structured subfields.
    This is a template—adjust regexes as needed.
    """
    ops_data = {
        "Depth": {"From": "", "To": ""},
        "Performance": {"Feet": "", "FPH": ""},
        "Rotation_Slide": {"Rotate": "", "Slide": ""},
        "Rotation_Time": {"Rotate Time": "", "Slide Time": ""},
        "GPM": "",
        "MTR RPM": "",
        "SPP": "",
        "DIFF": "",
        "WOB": "",
        "ROT RPM": "",
        "ON BTM TRQ": "",
        "OFF BTM TRQ": "",
        "GAS": {"Units": "", "Flare": ""},
        "MW": {"In": "", "Out": ""},
        "Targets": [],
        "Observations": []
    }

    # Split operations text by lines (or spaces) and attempt regex extraction.
    lines = ops_text.splitlines()
    for line in lines:
        l = line.strip()
        if not l:
            continue

        # Depth, e.g. "F/ 17,469' T/ 18,163'"
        depth_match = re.search(r"F/\s*([\d,']+)\s*T/\s*([\d,']+)", l, re.IGNORECASE)
        if depth_match:
            ops_data["Depth"]["From"] = depth_match.group(1)
            ops_data["Depth"]["To"] = depth_match.group(2)

        # Performance, e.g. "(694' @ 115 FPH)"
        perf_match = re.search(r"\(([\d,']+)\s*@\s*(\d+)\s*FPH\)", l, re.IGNORECASE)
        if perf_match:
            ops_data["Performance"]["Feet"] = perf_match.group(1)
            ops_data["Performance"]["FPH"] = perf_match.group(2)

        # Rotation/Slide percentages.
        rs_match = re.search(r"ROTATE\s*([\d.]+%)\s*/\s*SLIDE\s*([\d.]+%)", l, re.IGNORECASE)
        if rs_match:
            ops_data["Rotation_Slide"]["Rotate"] = rs_match.group(1)
            ops_data["Rotation_Slide"]["Slide"] = rs_match.group(2)

        # Rotation Time percentages.
        rt_match = re.search(r"ROTATE\s*TIME\s*([\d.]+%)\s*/\s*SLIDE\s*TIME\s*([\d.]+%)", l, re.IGNORECASE)
        if rt_match:
            ops_data["Rotation_Time"]["Rotate Time"] = rt_match.group(1)
            ops_data["Rotation_Time"]["Slide Time"] = rt_match.group(2)

        # Numeric parameters.
        gpm = re.search(r"GPM:\s*(\d+)", l, re.IGNORECASE)
        if gpm:
            ops_data["GPM"] = gpm.group(1)
        mtr = re.search(r"MTR\s*RPM:\s*(\d+)", l, re.IGNORECASE)
        if mtr:
            ops_data["MTR RPM"] = mtr.group(1)
        spp = re.search(r"SPP:\s*(\d+)", l, re.IGNORECASE)
        if spp:
            ops_data["SPP"] = spp.group(1)
        diff = re.search(r"DIFF:\s*([\d\-]+)", l, re.IGNORECASE)
        if diff:
            ops_data["DIFF"] = diff.group(1)
        wob = re.search(r"WOB:\s*(\d+)", l, re.IGNORECASE)
        if wob:
            ops_data["WOB"] = wob.group(1)
        rot_rpm = re.search(r"ROT\s*RPM:\s*(\d+)", l, re.IGNORECASE)
        if rot_rpm:
            ops_data["ROT RPM"] = rot_rpm.group(1)
        on_btm = re.search(r"ON\s*BTM\s*TRQ[:;]?\s*([\d\-K]+)", l, re.IGNORECASE)
        if on_btm:
            ops_data["ON BTM TRQ"] = on_btm.group(1)
        off_btm = re.search(r"OFF\s*BTM\s*TRQ[:;]?\s*([\d\-K]+)", l, re.IGNORECASE)
        if off_btm:
            ops_data["OFF BTM TRQ"] = off_btm.group(1)
        gas_units = re.search(r"GAS:\s*([\d,]+)\s*UNITS", l, re.IGNORECASE)
        if gas_units:
            ops_data["GAS"]["Units"] = gas_units.group(1)
        flare = re.search(r"(NO\s*FLARE|FLARE\s*ON|FLARE\s*\S+)", l, re.IGNORECASE)
        if flare:
            ops_data["GAS"]["Flare"] = flare.group(1)
        mw_in = re.search(r"MW\s*IN\s*([\d.]+)", l, re.IGNORECASE)
        if mw_in:
            ops_data["MW"]["In"] = mw_in.group(1)
        mw_out = re.search(r"OUT\s*([\d.]+)", l, re.IGNORECASE)
        if mw_out:
            ops_data["MW"]["Out"] = mw_out.group(1)
    
    
        # Capture Targets.
        if re.search(r"TARGET\s*#\d+", l, re.IGNORECASE):
            ops_data["Targets"].append(l)
        # Other lines become Observations if they don't match key patterns.
        if not re.search(r"(F/|ROTATE|GPM:|MTR RPM:|SPP:)", l, re.IGNORECASE):
            ops_data["Observations"].append(l)

    # Remove duplicate observations.
    ops_data["Observations"] = list(dict.fromkeys(ops_data["Observations"]))
    return ops_data


    #     # Capture Targets (lines with "TARGET #")
    #     if re.search(r"TARGET\s*#\d+", l, re.IGNORECASE):
    #         ops_data["Targets"].append(l)
    #     # Other lines become Observations if they don’t match key patterns.
    #     if not re.search(r"(F/|ROTATE|GPM:|MTR RPM:|SPP:)", l, re.IGNORECASE):
    #         ops_data["Observations"].append(l)

    # # Remove duplicate observations.
    # ops_data["Observations"] = list(dict.fromkeys(ops_data["Observations"]))
    # return ops_data

# ---------------------------------------------------------------------
# parse_all_rows_from_text
# ---------------------------------------------------------------------
def parse_all_rows_from_text(ocr_text):
    """
    Uses a lookahead regex to split the full OCR text into multiple row chunks.
    Each row chunk is then processed by splitting into tokens.
    Assumes the header for each row starts with two time stamps.
    """
    # Use lookahead to split where a new row starts.
    row_chunks = re.split(r"(?=\d{2}:\d{2}\s+\d{2}:\d{2})", ocr_text)
    rows = []
    for chunk in row_chunks:
        chunk = chunk.strip()
        if not chunk:
            continue
        # Split chunk into tokens.
        tokens = re.split(r"\s+", chunk)
        if len(tokens) < 10:
            continue  # Skip if too few tokens.
        # Assign fixed tokens based on our assumed structure.
        row_dict = {
            "From": tokens[0],
            "To": tokens[1],
            "Hours": tokens[2],
            "Depth Start": tokens[3],
            "Depth End": tokens[4],
            "Phase": " ".join(tokens[5:9]),  # tokens 5 to 8
            "Activity": tokens[9],
            "Operations Description": {}
        }
        # The remaining tokens form the raw operations description.
        operations_text = " ".join(tokens[10:])
        row_dict["Operations Description"] = parse_operations_description(operations_text)
        rows.append(row_dict)
    return rows

# ---------------------------------------------------------------------
# main_time_breakdown_pipeline
# ---------------------------------------------------------------------
def main_time_breakdown_pipeline():
    # Set your image path.
    img_path = "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_14.png"
    try:
        img = safe_read_image(img_path)
        logger.info("Image loaded successfully.")
    except Exception as e:
        logger.error(e)
        return

    # Preprocess the image.
    thresh_img = preprocess_image(img, debug=True)

    # OCR the entire image.
    full_text = single_box_ocr(thresh_img, debug=True)

    # Dynamically extract all rows using our lookahead-based parser.
    time_breakdown_list = parse_all_rows_from_text(full_text)
    if not time_breakdown_list:
        logger.error("No rows were detected. Please check the OCR output and header format.")
        return

    # Build final JSON output.
    final_output = {"TIME BREAKDOWN": time_breakdown_list}
    logger.info("===== FINAL TIME BREAKDOWN DATA =====")
    logger.info(json.dumps(final_output, indent=4))

    # Create a DataFrame (note: nested fields will be flattened).
    df = pd.json_normalize(final_output["TIME BREAKDOWN"])
    print("----- Extracted Time Breakdown DataFrame -----")
    print(df)

    # Save results as JSON and CSV.
    output_folder = "dbfs:/mnt/mini-proj-dd/final_time_breakdown_results"
    local_folder = output_folder.replace("dbfs:", "/dbfs")
    os.makedirs(local_folder, exist_ok=True)
    out_json = os.path.join(local_folder, "time_breakdown_data.json")
    with open(out_json, "w") as f:
        json.dump(final_output, f, indent=4)
    logger.info(f"JSON saved to {out_json}")
    out_csv = os.path.join(local_folder, "time_breakdown_data.csv")
    df.to_csv(out_csv, index=False)
    logger.info(f"CSV saved to {out_csv}")

if __name__ == "__main__":
    main_time_breakdown_pipeline()
